In [1]:
!pip install transformers
!pip install sentencepiece==0.1.91

/content/drive/.shortcut-targets-by-id/1L36Suaro13wM6_nraaxjqeqK-GXqfHLF/3차 프로젝트


In [5]:
import sys
sys.path.insert(0, '/content/drive/MyDrive/3차 프로젝트')

In [9]:
from t5model_train import CustomDataset, train, validate

In [ ]:
from transformers import T5Config, T5Tokenizer, T5ForConditionalGeneration
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader

import gc
gc.collect()
torch.cuda.empty_cache()

# pretrained model path
model_folder = 'google/mt5-small'

model = T5ForConditionalGeneration.from_pretrained(model_folder)
tokenizer = T5Tokenizer.from_pretrained(model_folder)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
model.to(device)

hyper-parameters

In [ ]:
config = T5Config()
config.MAX_LEN = 1024
config.SUMMARY_LEN = 150 
config.TRAIN_BATCH_SIZE = 4    # input batch size for training (default: 64)
config.VALID_BATCH_SIZE = 2    # input batch size for testing (default: 1000)
config.TRAIN_EPOCHS = 3        # number of epochs to train (default: 10)
config.VAL_EPOCHS = 1 
config.LEARNING_RATE = 1e-4    # learning rate (default: 0.01)
config.SEED = 42               # random seed (default: 42)

In [ ]:
train_params = {
        'batch_size': config.TRAIN_BATCH_SIZE,
        'shuffle': True,
        'num_workers': 0
        }

val_params = {
        'batch_size': config.VALID_BATCH_SIZE,
        'shuffle': False,
        'num_workers': 0
        }

optimizer = torch.optim.Adam(params =  model.parameters(), lr=config.LEARNING_RATE)

In [ ]:
!nvidia-smi

In [ ]:
import pandas as pd
train_dataset = pd.read_csv('/content/drive/MyDrive/3차 프로젝트/dataset/train.csv')[['document','label']]
validation_dataset = pd.read_csv('/content/drive/MyDrive/3차 프로젝트/dataset/valid.csv')[['document','label']]

train

In [ ]:
train_dataset.columns = ['ctext','text']
train_dataset.ctext = 'summarize: ' + train_dataset.ctext

training_set = CustomDataset(train_dataset, tokenizer, config.MAX_LEN, config.SUMMARY_LEN)

training_loader = DataLoader(training_set, **train_params)

for epoch in range(config.TRAIN_EPOCHS):
    print (1)
    train(epoch, tokenizer, model, device, training_loader, optimizer)

In [ ]:
tokenizer.save_pretrained('./pretrained_mt5_1125/')
model.save_pretrained('./pretrained_mt5_1125/')

valid(test)

In [ ]:
validation_dataset.columns = ['ctext','text']
validation_dataset.ctext = 'summarize: ' + validation_dataset.ctext

val_set = CustomDataset(validation_dataset, tokenizer, config.MAX_LEN, config.SUMMARY_LEN)

val_loader = DataLoader(val_set, **val_params)

for epoch in range(config.VAL_EPOCHS):
    predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
    final_df = pd.DataFrame({'Generated Text':predictions,'Actual Text':actuals})

final_df

----------------------------extract trained--------------------------------------

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

fine-tuning_model_folder = ''

extract_model = T5ForConditionalGeneration.from_pretrained(fine_tuning_model_folder)
extract_tokenizer = T5Tokenizer.from_pretrained(fine_tuning_model_folder)

In [ ]:
extract_model.to('cuda' if torch.cuda.is_available() else 'cpu')

-------------------hyper params------------------

In [ ]:
extract_config = T5Config()
extract_config.MAX_LEN = 1024
extract_config.SUMMARY_LEN = 150 
extract_config.TRAIN_BATCH_SIZE = 4    # input batch size for training (default: 64)
extract_config.TRAIN_EPOCHS = 3        # number of epochs to train (default: 10)
extract_config.LEARNING_RATE = 1e-4    # learning rate (default: 0.01)
extract_config.SEED = 42               # random seed (default: 42)

In [ ]:
extract_train_params = {
        'batch_size': config.TRAIN_BATCH_SIZE,
        'shuffle': True,
        'num_workers': 0
        }

extract_optimizer = torch.optim.Adam(params =  extract_model.parameters(), lr=extract_config.LEARNING_RATE)

-----------data----------------


In [ ]:
import pandas as pd
train_extract = pd.read_csv('/content/drive/MyDrive/3차 프로젝트/dataset/train.csv')[['document','label']]


----------------extract_train--------------

In [ ]:
train_extract.columns = ['ctext','text']
train_extract.ctext = 'summarize: ' + train_extract.ctext

training_extract_set = CustomDataset(train_extract, tokenizer, extract_config.MAX_LEN, extract_config.SUMMARY_LEN)

training_extract_loader = DataLoader(training_extract_set, **extract_train_params)

for epoch in range(extract_config.TRAIN_EPOCHS):
    print (1)
    train(epoch, extract_tokenizer, extract_model, device, training_extract_loader, extract_optimizer)

In [ ]:
model_save_path = './pretrained_mt5_1125/'

tokenizer.save_pretrained(model_save_path)
model.save_pretrained(model_save_path)